In [1]:
import urllib.request
import requests
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
import contractions
from sklearn.model_selection import train_test_split
from nltk.classify import NaiveBayesClassifier, DecisionTreeClassifier
from nltk.classify.util import accuracy
from sklearn.metrics import recall_score, precision_score, f1_score, auc, roc_curve, accuracy_score, precision_recall_curve
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import collections
from nltk.metrics.scores import (accuracy, precision, recall, f_measure)
from sklearn import tree

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html))'}
amazon_url ='https://www.amazon.com/product-reviews/B00HG1JMB4/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&showViewpoints=1&pageNumber='
ratings=[]
reviews = []

for i in range(0,400):
    url = amazon_url + str(i)
    response = requests.get(url, headers=headers).text
    soup = BeautifulSoup(response,'lxml')
    blocks = soup.find_all('div', attrs= {'class':"a-section review aok-relative"})
    for block in blocks:
        reviews.append(block.find('div', attrs= {'class':'a-row a-spacing-small review-data'}))
        ratings.append(str(block.find('i', attrs= {'data-hook':'review-star-rating'}))[36:37])
        
new = re.compile('\s+')
review = [new.sub(' ',a.text).strip() for a in reviews]

df = (pd.DataFrame({'ratings':ratings,'review':review})[10:410])
df.to_csv('counter_rev.csv')

In [2]:
df_real= pd.read_csv('Countour_reviews_new.csv')# Inserted new columns with counterfeit and no complaint. 

In [8]:
from sklearn.utils import shuffle
df = shuffle(df_real, random_state=0)

X =df['review'].values.astype(str)
Y = df['Review_label'].values

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.30, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(280,) (280,)
(120,) (120,)


In [10]:
def txt_preprocess(s):
    r=[]
    words = word_tokenize(str(s))#tokenizes reviews
    clean_words = [word.lower() for word in words if word not in set(string.punctuation)]#lwoer case
    english_stops = set(stopwords.words('english'))#gets rid of stop words
    clean_words = [word for word in clean_words if word not in english_stops]
    clean_words = ' '.join(word for word in clean_words if not word.isdigit())# egts rid of numbers
    characters_to_remove = ['...', '', '’', "n't", "'s'", '”', "", ' ', '  ','..','.',str("'s"),".i",'“','``']
    clean_words = [word for word in clean_words.split() if word not in set(characters_to_remove)] 
    wordnet_lemmatizer = WordNetLemmatizer()
    lemma_list = [wordnet_lemmatizer.lemmatize(word) for word in clean_words]
    return lemma_list

In [11]:
#returns tuple
def bag_of_ngrams(s):
    return dict([(ngram, True) for ngram in s])


nb_train_feats = []
nb_test_feats = []

for i in range(0,len(X_train)):
    nb_train_feats.append(tuple([bag_of_ngrams(txt_preprocess(X_train[i])),Y_train[i]]))

for i in range(0,len(X_test)):
    nb_test_feats.append(tuple([bag_of_ngrams(txt_preprocess(X_test[i])),Y_test[i]]))

In [12]:
nb_classifier = NaiveBayesClassifier.train(nb_train_feats)
y_true=[]
y_score = []

for i, (feats, label_true) in enumerate(nb_test_feats):
    label_predicted = nb_classifier.classify(feats)
    y_true.append(label_true)
    y_score.append(label_predicted)
from sklearn.metrics import recall_score, precision_score, f1_score, auc, roc_curve, accuracy_score, precision_recall_curve
# Precision-Recall AUC
precision, recall, _ = precision_recall_curve(y_true, y_score, pos_label=1)

print('Model accuracy: ', accuracy_score(y_true, y_score))

# ROC AUC
fpr, tpr, _ = roc_curve(y_true, y_score, pos_label=1)
roc_auc = auc(fpr, tpr)
print("ROC AUC: ", roc_auc)

pr_auc = auc(recall, precision)
print("Precision-Recall AUC: ", pr_auc)

print('Recall: ', recall_score(y_true, y_score))
print('Precision: ', precision_score(y_true, y_score))
print('F-measure: ', f1_score(y_true, y_score))

Model accuracy:  0.75
ROC AUC:  0.7666666666666667
Precision-Recall AUC:  0.675
Recall:  0.8
Precision:  0.5
F-measure:  0.6153846153846154


In [13]:
nb_classifier.show_most_informative_features(10)

Most Informative Features
                    fake = True                1 : 0      =     32.6 : 1.0
                    real = True                1 : 0      =     25.4 : 1.0
                received = True                1 : 0      =     13.3 : 1.0
                    find = True                1 : 0      =     10.9 : 1.0
                  return = True                1 : 0      =     10.9 : 1.0
                  actual = True                1 : 0      =     10.9 : 1.0
            disappointed = True                1 : 0      =      9.8 : 1.0
                   great = True                0 : 1      =      8.6 : 1.0
                 package = True                1 : 0      =      8.5 : 1.0
               authentic = True                1 : 0      =      8.5 : 1.0


In [14]:
tvec = TfidfVectorizer()

tvec.set_params(max_df=0.95, 
                min_df=0.05,
                max_features=3000)

lr = LogisticRegression()
dt = tree.DecisionTreeClassifier()

LR_Pipeline = Pipeline([
    ('vectorizer', tvec),
    ('classifier', lr)
])

DTree_Pipeline = Pipeline([
    ('vectorizer', tvec),
    ('dt_classifier', dt)
])

LR_sentiment_model = LR_Pipeline.fit(X_train, Y_train)
DT_sentiment_model = DTree_Pipeline.fit(X_train, Y_train)

y_pred_lr = LR_sentiment_model.predict(X_test)
y_pred_dt = DT_sentiment_model.predict(X_test)

# ROC AUC
fpr, tpr, _ = roc_curve(y_true, y_pred_lr, pos_label=1)
roc_auc_lr = auc(fpr, tpr)

fpr2, tpr2, _2 = roc_curve(y_true, y_pred_dt, pos_label=1)
roc_auc_dt = auc(fpr2, tpr2)

print("LR ROC AUC: ", roc_auc_lr)
print('LR Model Accuracy: ', accuracy_score(Y_test,y_pred_lr))
print('LR Recall: ', recall_score(Y_test, y_pred_lr))
print('LR Precision: ', precision_score(Y_test, y_pred_lr))
print('LR F1 :', f1_score(Y_test, y_pred_lr))
print()
print("DTree ROC AUC: ", roc_auc_dt)
print('DTree Model Accuracy: ', accuracy_score(Y_test,y_pred_dt))
print('DTree Recall: ', recall_score(Y_test, y_pred_dt))
print('DTree Precision: ', precision_score(Y_test, y_pred_dt))
print('DTree F1 :', f1_score(Y_test, y_pred_dt))

LR ROC AUC:  0.7
LR Model Accuracy:  0.85
LR Recall:  0.4
LR Precision:  1.0
LR F1 : 0.5714285714285715

DTree ROC AUC:  0.6611111111111112
DTree Model Accuracy:  0.7583333333333333
DTree Recall:  0.4666666666666667
DTree Precision:  0.5185185185185185
DTree F1 : 0.4912280701754386


C:\Users\jacke\Downloads\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
